In [1]:
import polars as pl
import sys

sys.path.append("../../")
from src.utils import *

In [2]:
pl.Config.load_from_file("../../polars_cfg.json")

In [3]:
all_data_path = "../../data/clean_all_data_merged.csv"
errors_path = "../../data/clean_errors_merged.csv"
feedback_path = "../../data/clean_feedback_data_merged.csv"
incomplete_path = "../../data/clean_incomplete_data_merged.csv"
summary_path = "../../data/clean_summary_data_merged.csv"
demographics_path = "../../data/clean_demographics_merged.csv"

df_all = pl.read_csv(all_data_path)
df_errors = pl.read_csv(errors_path)
df_feedback = pl.read_csv(feedback_path)
df_incomplete = pl.read_csv(incomplete_path)
df_summary = pl.read_csv(summary_path)

# data


In [4]:
reordered_columns = [
    # Experiment metadata
    "exp_name",
    "task_type",
    # Identifiers and metadata
    "hashed_id",
    "joint_id_task",
    # Task-related information
    "task_name",
    "task_number",
    "is_tutorial",
    # Time-related information
    "time",
    # Task performance
    "attempt_number",
    "num_actions",
    "solved",
    "done",
    # Input and output data
    "test_input_grid",
    "test_input_size_x",
    "test_input_size_y",
    "test_output_grid",
    "test_output_size_x",
    "test_output_size_y",
    # User actions
    "action",
    "action_x",
    "action_y",
    "select_loc",
    "selected_data",
    "selected_symbol",
    "selected_tool",
    "copy_paste_data",
    # User solutions
    "first_written_solution",
    "last_written_solution",
    # Withdrawal information
    "withdraw",
    "withdraw_reason",
    "withdraw_comment",
    # Demographic information
    "age",
    "gender",
    "race",
    "education_level",
    "household_income",
    "normal_vision",
    "color_blind",
    "fluent_english",
]

In [5]:
def standardize_test_input_size(df):
    df = (
        df.with_columns(
            pl.col("test_input_size")
            .str.strip_chars("[]")
            .str.split(",")
            .list.to_struct(fields=["test_input_size_x", "test_input_size_y"])
        )
        .unnest("test_input_size")
        .with_columns(
            [
                pl.col("test_input_size_x").str.strip_chars(" ").cast(pl.Int32),
                pl.col("test_input_size_y").str.strip_chars(" ").cast(pl.Int32),
            ]
        )
    )
    df = (
        df.with_columns(
            pl.col("test_output_size")
            .map_elements(lambda x: eval(x), return_dtype=pl.List(pl.String))
            .list.to_struct(fields=["test_output_size_x", "test_output_size_y"])
        )
        .unnest("test_output_size")
        .with_columns(
            [
                pl.col("test_output_size_x").str.strip_chars(" ").cast(pl.Int32),
                pl.col("test_output_size_y").str.strip_chars(" ").cast(pl.Int32),
            ]
        )
    )
    return df

In [6]:
df_incomplete = standardize_test_input_size(df_incomplete)
df_all = standardize_test_input_size(df_all)

In [7]:
# output time in standard format
df_all = df_all.with_columns(
    pl.col("time").map_elements(parse_mixed_datetime, return_dtype=pl.Datetime),
)
df_incomplete = df_incomplete.with_columns(
    pl.col("time").map_elements(parse_mixed_datetime, return_dtype=pl.Datetime),
)

In [8]:
# missed complete
missed_complete = df_incomplete.select(
    pl.all()
    .top_k_by(["num_actions"], k=1)
    .over(["joint_id_task"], mapping_strategy="explode")
).filter(
    pl.col("action").is_in(
        ["no_last_description", "write_description", "write_last_description", "submit"]
    )
)
missed_complete.group_by("hashed_id").agg(
    pl.len().alias("count"), pl.first("exp_name")
).filter(
    pl.col("count") >= 5
)  # there are no missed complete data

hashed_id,count,exp_name
str,u32,str
"""62ef141694b004752bc4d6352148b6aa""",6,"""expv1"""
"""c3cd77edefb6b3c9bffcc4ae6940d182""",7,"""expv1"""
"""945cd915c78988165db7af93206ec325""",7,"""expv1"""
"""00e515b84bc3a0e4c555e208aded5f96""",5,"""expv1"""
"""f2225b6c4d0910cf5b10543f22de2dd5""",5,"""expv1"""


In [9]:
# edge case not accounted for by initial filtering (all happen to be in expv0 or expv1)
edge_cases = (
    df_all.filter((pl.col("done").is_null()))
    .unique("joint_id_task")
    .select("hashed_id")
)
missed_incomplete = (
    df_all.filter(pl.col("hashed_id").is_in(edge_cases))
    .select(
        pl.all()
        .top_k_by(["num_actions"], k=1)
        .over(["joint_id_task"], mapping_strategy="explode")
    )
    .group_by("hashed_id")
    .agg(pl.len().alias("count"), pl.first("exp_name"))
    .filter(pl.col("count") < 10)
)
missed_incomplete = df_all.filter(
    pl.col("hashed_id").is_in(missed_incomplete.select("hashed_id"))
)

In [10]:
df_all = df_all.filter(
    ~pl.col("hashed_id").is_in(missed_incomplete.select("hashed_id"))
)

In [11]:
missed_incomplete = missed_incomplete.select(reordered_columns)
df_incomplete = df_incomplete.select(reordered_columns)

In [12]:
df_incomplete = df_incomplete.with_columns(
    pl.when(pl.col("withdraw").str.to_lowercase() == "true")
    .then(True)
    .when(pl.col("withdraw").str.to_lowercase() == "false")
    .then(False)
    .otherwise(None)
    .alias("withdraw")
    .cast(pl.Boolean),
    pl.col("age").cast(pl.Float32),
    pl.lit(False).alias("complete"),
    pl.when(True).then(False).alias("done"),  # force done to false
)

missed_incomplete = missed_incomplete.with_columns(
    pl.col("withdraw").cast(pl.Boolean),
    pl.col("age").cast(pl.Float32),
    pl.lit(False).alias("complete"),
    pl.lit(True).alias("done"),
)

df_incomplete = pl.concat([df_incomplete, missed_incomplete])

# Get the hashed_id and attempt_number combinations where action is submit
submit_rows = df_incomplete.filter(
    (
        pl.col("action").is_in(
            [
                "no_last_description",
                "write_description",
                "write_last_description",
            ]
        )
    )
).select(["joint_id_task", "attempt_number"])

# Update done to True for all rows matching those combinations
df_incomplete = df_incomplete.with_columns(
    pl.when(
        pl.struct(["joint_id_task", "attempt_number"]).is_in(
            submit_rows.select(["joint_id_task", "attempt_number"])
        )
    )
    .then(True)
    .otherwise(False)
    .alias("done")
)


df_all = df_all.select(reordered_columns).with_columns(
    pl.col("withdraw").cast(pl.Boolean),
    pl.col("age").cast(pl.Float32),
    pl.lit(True).alias("complete"),
    pl.lit(True).alias("done"),
)

Got rid of:

- feedback
- indexOf
- phase
- template
- templates
- viewTime
- tutorial_response
- beginHit
- endHit


In [13]:
df_all_ = pl.concat([df_all, df_incomplete])

In [14]:
df_all_.rename({"num_actions": "action_id"}).write_csv(
    "/Users/solimlegris/Projects/h-arc-osf/data/data.csv"
)

# clean summary


In [15]:
df_summary = get_summary(df_all_, verbose=True)

Filtered out 62/8899 participant task attempts


In [16]:
reordered_columns_summary = [
    # Experiment metadata
    "exp_name",
    "task_type",
    # Identifiers and metadata
    "hashed_id",
    "joint_id_task",
    # Task-related information
    "task_name",
    "task_number",
    # Task performance
    "attempt_number",
    "num_actions",
    "solved",
    # Output data
    "test_output_grid",
    # User solutions
    "first_written_solution",
    "last_written_solution",
    # complete
    "complete",
]

In [17]:
df_summary = df_summary.select(reordered_columns_summary)
df_summary.head()

exp_name,task_type,hashed_id,joint_id_task,task_name,task_number,attempt_number,num_actions,solved,test_output_grid,first_written_solution,last_written_solution,complete
str,str,str,str,str,i64,i64,i64,bool,str,str,str,bool
"""expv2""","""training""","""59e36641c3391529505bdf7af902cec0""","""59e36641c3391529505bdf7af902cec0_32597951.jso…","""32597951.json""",1,1,133,false,"""|10010010010010010|01001001001001001|00100100…","""YES DONE EASILY""","""INTERESTING TO COMPLETE""",true
"""expv2""","""training""","""59e36641c3391529505bdf7af902cec0""","""59e36641c3391529505bdf7af902cec0_32597951.jso…","""32597951.json""",1,2,151,false,"""|10010010010010010|01001001001001001|00100100…","""YES DONE EASILY""","""INTERESTING TO COMPLETE""",true
"""expv2""","""training""","""59e36641c3391529505bdf7af902cec0""","""59e36641c3391529505bdf7af902cec0_32597951.jso…","""32597951.json""",1,3,182,false,"""|10010010010010010|01001001001001001|00100100…","""YES DONE EASILY""","""INTERESTING TO COMPLETE""",true
"""expv2""","""training""","""59e36641c3391529505bdf7af902cec0""","""59e36641c3391529505bdf7af902cec0_321b1fc6.jso…","""321b1fc6.json""",2,1,43,false,"""|0000000000|0440004400|3433034330|0030000300|…","""SUPER EXPERIMENT KEEP COMPELETE THE TASK""","""INTERESTING TO COMPLE THE TASKS""",true
"""expv2""","""training""","""59e36641c3391529505bdf7af902cec0""","""59e36641c3391529505bdf7af902cec0_321b1fc6.jso…","""321b1fc6.json""",2,2,53,true,"""|0000000000|0440004400|3433034330|0030000300|…","""SUPER EXPERIMENT KEEP COMPELETE THE TASK""","""INTERESTING TO COMPLE THE TASKS""",true


In [18]:
df_summary.write_csv("/Users/solimlegris/Projects/h-arc-osf/data/summary_data.csv")

# clean errors


In [19]:
df_errors = get_errors(df_summary)
df_errors.head()

task_name,test_output_grid,count,task_type,hashed_output_grid
str,str,u32,str,str
"""f25fbde4.json""","""|000000000|444444444|444444444|444444444|0000…",1,"""training""","""fa0872eac83daf7dfc5c33fa4bc099bc"""
"""fcc82909.json""","""|0000000000|1100002900|2100001600|3304703300|…",4,"""training""","""132c75dd645b5ce87f054175c766b295"""
"""913fb3ed.json""","""|1110000000000000|1310000000000000|1110000000…",1,"""training""","""42bfc6a865dc372f0452e8074dd27b6e"""
"""ac3e2b04.json""","""|002000020002000020|033300111002000020|132311…",1,"""evaluation""","""70a13709858b3a68c7b31b524a7f7d88"""
"""af22c60d.json""","""|885989585655225665225565859895|8998898566562…",1,"""evaluation""","""429693fd10a5269c09dad69c4e53ce5b"""


In [20]:
df_errors.select(["task_name", "task_type", "test_output_grid", "count"]).write_csv(
    "/Users/solimlegris/Projects/h-arc-osf/data/incorrect_submissions.csv"
)

# clean feedback


In [21]:
reordered_columns_feedback = [
    # Experiment metadata
    "exp_name",
    "task_type",
    # Identifiers and metadata
    "hashed_id",
    # feedback
    "feedback",
]

In [22]:
df_feedback = df_feedback.select(reordered_columns_feedback)
df_feedback.head()

exp_name,task_type,hashed_id,feedback
str,str,str,str
"""expv0""","""training""","""195f0749824286cb4b24f431004a3b87""","""Interesting. I can't really explain my strate…"
"""expv0""","""training""","""bf66a164234034d4709666dc364c8d55""","""this was fun - thanks! """
"""expv0""","""training""","""2b06355cea411af7a6fa212c90eb3c6d""","""Thank you."""
"""expv0""","""training""","""19f69ac203258689f6823e2c49cea6b7""","""Very interesting."""
"""expv0""","""training""","""37289334396727fffe878187ca738a35""","""The first 5 were jacked up. The controls didn…"


In [23]:
df_feedback.write_csv("/Users/solimlegris/Projects/h-arc-osf/survey/feedback.csv")

# demographics


In [24]:
df_demographics = df_all_.select(
    "exp_name",
    "task_type",
    "hashed_id",
    "age",
    "gender",
    "race",
    "education_level",
    "household_income",
    "normal_vision",
    "color_blind",
    "fluent_english",
).unique("hashed_id")
df_demographics.head()

exp_name,task_type,hashed_id,age,gender,race,education_level,household_income,normal_vision,color_blind,fluent_english
str,str,str,f32,str,str,str,str,str,str,str
"""expv6_4""","""evaluation""","""bd5156b61b4c1e8436fd9c6f4b56e518""",53.0,"""Man""","""Asian""","""Graduate Degree (MA/MS/MPhil/Other)""","""$40,000–$59,999""","""Yes""","""No""","""Yes"""
"""expv2""","""training""","""fb11b459ca7e8efc451096a15f906516""",36.0,"""Man""","""White""","""Undergraduate Degree (BA/BS/Other)""","""$20,000–$39,999""","""Yes""","""No""","""Yes"""
"""expv1""","""training""","""2e0aa57f3c0e4f5995206a9446937e3b""",54.0,"""female""",null,null,null,null,null,null
"""expv6_6""","""evaluation""","""cae2d6af70f490bec3c0c3539002ac67""",35.0,"""Man""","""White""","""Technical/Community College""","""Less than $20,000""","""Yes""","""No""","""Yes"""
"""expv6_4""","""evaluation""","""660da56639d6c1adf6d11cb40caa07a2""",44.0,"""Man""","""White""","""Undergraduate Degree (BA/BS/Other)""","""$100,000–$199,999""","""Yes""","""No""","""Yes"""


In [25]:
df_demographics.write_csv(
    "/Users/solimlegris/Projects/h-arc-osf/survey/demographics.csv"
)

# withdraw data


In [26]:
reordered_columns_withdraw = [
    # Experiment metadata
    "exp_name",
    "task_type",
    # Identifiers and metadata
    "hashed_id",
    "withdraw",
    "withdraw_reason",
    "withdraw_comment",
]

In [27]:
df_withdraw = df_all_.select(reordered_columns_withdraw)
df_withdraw = df_withdraw.unique("hashed_id")
df_withdraw.head()

exp_name,task_type,hashed_id,withdraw,withdraw_reason,withdraw_comment
str,str,str,bool,str,str
"""expv6_4""","""evaluation""","""28dfd3dab1bd5825869467245a1af7a8""",false,null,null
"""expv4""","""training""","""e58568496d2090ed40f1c1614a51265c""",false,null,null
"""expv4""","""training""","""faad81dd4a698db576d23febbfe9812b""",false,null,null
"""expv0""","""training""","""2204a682efe5fa9182c972e992355d99""",null,null,null
"""expv6_4""","""evaluation""","""832fcaa84fd16b40b5d030c1a50418ad""",true,"""I do not understand what I am supposed to do.""","""The examples given as inputs (especially the …"


In [28]:
df_withdraw.write_csv("/Users/solimlegris/Projects/h-arc-osf/survey/withdraw.csv")